# SQUARE DONUT

**by Serhat Çevikel**

You are supposed to write a function named "square_donut" that takes two arguments n and x:
- That creates a square matrix of size n
- The outer x squares of the matrix should have 1, while the square "hole" in the middle should be all zeros
- If x is greater than or equal to the half of n, it should throw a message as "edge too wide" and stop

So:
```R
square_donut(7, 2)
```
returns
```
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1
```

```R
square_donut(6, 3)
```
returns
```
'edge too wide'
```

## SOLUTION

In [ ]:
square_donut <- function(n, x)
{
    if (x >= n / 2)
    {
        return("edge too wide")
    }
    
    mat1 <- matrix(1, nrow = n, ncol = n) # create matrix of 1's
    startx <- x + 1 # start index of 0's
    endx <- n - x # end index of 0's
    mat1[startx:endx, startx:endx] <- 0 # create the hole of 0's
    
    return(mat1)
}

In [ ]:
square_donut(16, 2)

# RIDDLE DOWN THE MATRIX

You are given a matrix generator function as such:

```R
matrix_gen <- function(maxx = 100, nr = 5, nc = 5)
{
    matrix(sample(maxx, nr * nc, replace = T), nrow = nr) 
}
```

```R
Using this matrix, you will write a function as such:

max_path <- function(matt)
{

...

}
```


That will return the maximum sum to be attained when a path is selected from the first row to the end, in each step of which you can select any of the adjacent cells (straight down, down left, down right) on the next row.

Let's say, for a random 3x3 matrix of 1:10 values:

```R
mattt <- matrix_gen(10, 3, 3)
mattt
     [,1] [,2] [,3]
[1,]    9    3    4
[2,]    4    4    3
[3,]    6    5    9
```
```R
max_path(mattt)
[1] 22
```

(I guess when you follow the primary diagon)


Or when:

```R
mattt <- matrix_gen(10, 5, 5)
mattt
     [,1] [,2] [,3] [,4] [,5]
[1,]    2    8    2   10    9
[2,]    3    5    5   10    3
[3,]    9    8   10   10   10
[4,]    8    5    6    7    5
[5,]    5    1    6    9    1
```

The result is:

```
max_path(mattt)
[1] 46
```

(I guess it is the 4th column)

HINT: You may make use of pmax() function or just apply()

## SOLUTION

In [ ]:
matrix_gen <- function(maxx = 100, nr = 5, nc = 5)
{
    matrix(sample(maxx, nr * nc, replace = T), nrow = nr)
}

max_path <- function(matt)
{
    # we will track a vector length of which is equal to the number of the rows and
    # that shows the max sums for each cell in the current row up to that row
    # we take the first row as the initial value for this vector (the values show the maximum sum for each column up to that point!)
    sums_vec <- matt[1,]

    # get the dimensions of the original matrix. can also be done with dim()
    nc <- ncol(matt)
    nr <- nrow(matt)
    
    # across rows, starting from the second one
    # on the first row the max values up to that row are the values themselves
    for (roww in 2:nr) 
    {
        # takes three vector:
        # the original sums_vec, the sums_vec offset to left and offset to right
        # this is done by adding a zero to left/trimming from right 
        # or adding a zero to right/trimming from left
        # why? we can come to a cell from straight up, left up or right up
        # so for each cell in the row we should the select the max of these three values
        # pmax does it pairwise (or triple wise in this case)
        # this is the critical part of the code
        max_of_sums_vec <- pmax(c(0, sums_vec[-nc]), sums_vec, c(sums_vec[-1], 0))

        # update the sums vec: add the maximum sums vector (that shows the maximum sums up to that row for each cell)
        # to the current row
        sums_vec <- matt[roww,] + max_of_sums_vec
    }

    # we have come to the last row
    # now just take the maximum of the last sums_vec
    maxval <- max(sums_vec)

    return(maxval)

}

In [ ]:
mat1 <- matrix_gen(10, 3, 3)
mat1
max_path(mat1)